In [2]:
"""
以启发式思维实现的、基于concept map的学习知识概念推荐算法
mk0，尝试完再转为py代码实现
"""
import os
import networkx as nx

In [31]:
class NodeType:
    SCOPE = 'SCOPE'
    MODULE = 'MODULE'
    PACKAGE = 'PACKAGE'
    CLASS = 'CLASS'
    INTERFACE = 'INTERFACE'
    ENUM = 'ENUM'
    EXCEPTION = 'EXCEPTION'
    ERROR = 'ERROR'
    ANNOTATION = 'ANNOTATION'
    FIELD = 'FIELD'
    METHOD = 'METHOD'
    ENUM_CONSTANT = 'ENUM_CONSTANT'
    OPTIONAL_ELEMENT = 'OPTIONAL_ELEMENT'
    CONSTRUCTOR = 'CONSTRUCTOR'
    OTHER = 'OTHER'
class EdgeType:
    INCLUDE = 'INCLUDE'
    EXPORT = 'EXPORT'
    REQUIRE = 'REQUIRE'
    INHERIT = 'INHERIT'
    IMPLEMENT = 'IMPLEMENT'
    REFERENCE_IN_DESCRIPTION = 'REFERENCE_IN_DESCRIPTION' #reference when talking about the description
    PARAMETER = 'PARAMETER' #this function use an object of this class as parameter
    RETURN_TYPE = 'RETURN_TYPE' #this function return an object of this class
    FIELD_IS_TYPE = 'FIELD_IS_TYPE' #the field's type is this class
    NESTED_CLASS = 'NESTED_CLASS' #the class is a nested class of another class, can be identified with Enclosing class
    ATTACH_ANNOTATION = 'ATTACH_ANNOTATION' #method or field or class is attached with an annotation
    THROWS = 'THROWS' #a method throw an exception or an error
    ALSO_SEE = 'ALSO_SEE' #two related concepts, marked as also see in the doc

class_level_node_types = [
    NodeType.ANNOTATION, 
    NodeType.CLASS, 
    NodeType.ENUM, 
    NodeType.ERROR,
    NodeType.EXCEPTION,
    NodeType.INTERFACE
]


In [32]:
cm_path = 'F:/SOworkspace/backup/concept_map_javadoc20200801.gexf'
concept_map = nx.read_gexf(cm_path)
package_nodes = [node for node in concept_map.nodes if 'Ntype' in concept_map.nodes[node].keys() and concept_map.nodes[node]['Ntype'] == 'PACKAGE']
package_nodes

['java.net.http',
 'java.rmi',
 'java.sql',
 'jdk.dynalink',
 'jdk.incubator.foreign',
 'jdk.jfr',
 'jdk.jshell',
 'jdk.management.jfr',
 'jdk.net',
 'java.io',
 'java.lang',
 'java.lang.annotation',
 'java.lang.constant',
 'java.lang.invoke',
 'java.lang.module',
 'java.lang.ref',
 'java.lang.reflect',
 'java.lang.runtime',
 'java.math',
 'java.net',
 'java.net.spi',
 'java.nio',
 'java.nio.channels',
 'java.nio.channels.spi',
 'java.nio.charset',
 'java.nio.charset.spi',
 'java.nio.file',
 'java.nio.file.attribute',
 'java.nio.file.spi',
 'java.security',
 'java.security.cert',
 'java.security.interfaces',
 'java.security.spec',
 'java.text',
 'java.text.spi',
 'java.time',
 'java.time.chrono',
 'java.time.format',
 'java.time.temporal',
 'java.time.zone',
 'java.util',
 'java.util.concurrent',
 'java.util.concurrent.atomic',
 'java.util.concurrent.locks',
 'java.util.function',
 'java.util.jar',
 'java.util.regex',
 'java.util.spi',
 'java.util.stream',
 'java.util.zip',
 'javax.cry

In [5]:
def get_related_concepts_submap(concept_map : nx.DiGraph, concept_node : str):
    submap = concept_map.subgraph(list(nx.dfs_postorder_nodes(concept_map, concept_node)))
    #nx.write_gexf(submap, os.path.join(TEMP_FILE_STORE_PATH, str(concept_node) + '_cache.gexf'))
    return submap

In [10]:
 """
 以java.util.concurrent为学习目标case展开尝试
 """
target = "java.util.concurrent"
sub_concept_map = get_related_concepts_submap(concept_map, target)

In [34]:
class_level_sub_concept_map = sub_concept_map.subgraph([
    node for node in sub_concept_map
    if 'Ntype' in sub_concept_map.nodes[node].keys() and 
    sub_concept_map.nodes[node]['Ntype'] in class_level_node_types
])

In [43]:
candidate_nodes = [node for node in sub_concept_map.nodes if 
                   'Ntype' in sub_concept_map.nodes[node].keys() and 
                   sub_concept_map.nodes[node]['Ntype'] in class_level_node_types and 
                   sub_concept_map.in_degree(node) < 15 and 
                   sub_concept_map.out_degree(node) > 5 and
                   'concurrent' in node]
"""candidate_nodes = [node for node in class_level_sub_concept_map.nodes if 
                   sub_concept_map.in_degree(node) < 5 and 
                   sub_concept_map.out_degree(node) > 5 and
                   'concurrent' in node]"""

"candidate_nodes = [node for node in class_level_sub_concept_map.nodes if \n                   sub_concept_map.in_degree(node) < 5 and \n                   sub_concept_map.out_degree(node) > 5 and\n                   'concurrent' in node]"

In [44]:
candidate_nodes

['api/java.base/java/util/concurrent/Executors.html',
 'api/java.base/java/util/concurrent/locks/Condition.html',
 'api/java.base/java/util/concurrent/ConcurrentMap.html',
 'api/java.base/java/util/concurrent/atomic/AtomicLong.html',
 'api/java.base/java/util/concurrent/ConcurrentLinkedQueue.html',
 'api/java.base/java/util/concurrent/atomic/LongAdder.html',
 'api/java.base/java/util/concurrent/Exchanger.html',
 'api/java.base/java/util/concurrent/Phaser.html',
 'api/java.base/java/util/concurrent/SubmissionPublisher.html',
 'api/java.base/java/util/concurrent/ThreadLocalRandom.html',
 'api/java.base/java/util/concurrent/SynchronousQueue.html',
 'api/java.base/java/util/concurrent/ThreadPoolExecutor.AbortPolicy.html',
 'api/java.base/java/util/concurrent/CyclicBarrier.html',
 'api/java.base/java/util/concurrent/CountedCompleter.html',
 'api/java.base/java/util/concurrent/Flow.html',
 'api/java.base/java/util/concurrent/CopyOnWriteArraySet.html',
 'api/java.base/java/util/concurrent/Rec

In [48]:
candidate_in_out = [(node, class_level_sub_concept_map.in_degree(node), class_level_sub_concept_map.out_degree(node)) for node in candidate_nodes]
sorted(candidate_in_out, key=lambda n: n[2] + n[1], reverse=True)

[('api/java.base/java/util/concurrent/ThreadPoolExecutor.html', 6, 16),
 ('api/java.base/java/util/concurrent/ForkJoinPool.html', 4, 11),
 ('api/java.base/java/util/concurrent/FutureTask.html', 7, 6),
 ('api/java.base/java/util/concurrent/Executors.html', 5, 6),
 ('api/java.base/java/util/concurrent/locks/Lock.html', 7, 4),
 ('api/java.base/java/util/concurrent/SynchronousQueue.html', 3, 7),
 ('api/java.base/java/util/concurrent/CountedCompleter.html', 2, 8),
 ('api/java.base/java/util/concurrent/ScheduledThreadPoolExecutor.html', 2, 8),
 ('api/java.base/java/util/concurrent/PriorityBlockingQueue.html', 1, 9),
 ('api/java.base/java/util/concurrent/CyclicBarrier.html', 3, 6),
 ('api/java.base/java/util/concurrent/locks/ReentrantReadWriteLock.html',
  1,
  8),
 ('api/java.base/java/util/concurrent/ArrayBlockingQueue.html', 2, 7),
 ('api/java.base/java/util/concurrent/AbstractExecutorService.html', 2, 6),
 ('api/java.base/java/util/concurrent/LinkedBlockingQueue.html', 1, 7),
 ('api/java.

In [46]:
class_level_sub_concept_map.in_edges('api/java.base/java/util/concurrent/Executors.html')

InEdgeDataView([('api/java.base/java/util/concurrent/Callable.html', 'api/java.base/java/util/concurrent/Executors.html'), ('api/java.base/java/util/concurrent/Executor.html', 'api/java.base/java/util/concurrent/Executors.html'), ('api/java.base/java/util/concurrent/ExecutorService.html', 'api/java.base/java/util/concurrent/Executors.html'), ('api/java.base/java/util/concurrent/ScheduledExecutorService.html', 'api/java.base/java/util/concurrent/Executors.html'), ('api/java.base/java/util/concurrent/ThreadPoolExecutor.html', 'api/java.base/java/util/concurrent/Executors.html')])

In [52]:
important_nodes = set()
for candidate, ind, outd in candidate_in_out:
    dfs_nodes = nx.dfs_postorder_nodes(class_level_sub_concept_map, candidate, depth_limit = 2)
    for temp_node in dfs_nodes:
        important_nodes.add(temp_node)
list(important_nodes)

['api/java.base/java/util/concurrent/ConcurrentHashMap.KeySetView.html',
 'api/java.base/java/util/Map.html',
 'api/java.base/java/util/concurrent/ConcurrentHashMap.html',
 'api/java.base/java/util/concurrent/Flow.Subscription.html',
 'api/java.base/java/util/concurrent/DelayQueue.html',
 'api/java.base/java/util/concurrent/ThreadLocalRandom.html',
 'api/java.base/java/lang/AutoCloseable.html',
 'api/java.base/java/util/concurrent/Phaser.html',
 'api/java.base/java/util/concurrent/Future.html',
 'api/java.base/java/util/concurrent/locks/Condition.html',
 'api/java.base/java/util/concurrent/RecursiveTask.html',
 'api/java.base/java/util/concurrent/ThreadPoolExecutor.DiscardPolicy.html',
 'api/java.base/java/util/concurrent/atomic/LongAdder.html',
 'api/java.base/java/util/concurrent/AbstractExecutorService.html',
 'api/java.base/java/lang/InterruptedException.html',
 'api/java.base/java/util/concurrent/LinkedTransferQueue.html',
 'api/java.base/java/util/concurrent/locks/ReentrantLock.h

In [58]:
final_important_nodes = set()
for important_node in important_nodes:
    con_preds = [p for p in concept_map.pred[important_node] if 'concurrent' in p]
    preds = list(concept_map.pred[important_node])
    if len(con_preds) * 2 < len(preds):
        continue
    final_important_nodes.add(important_node)

['api/java.base/java/util/concurrent/ConcurrentHashMap.KeySetView.html',
 'api/java.base/java/util/AbstractQueue.html',
 'api/java.base/java/util/concurrent/ExecutorService.html',
 'api/java.base/java/util/concurrent/CopyOnWriteArraySet.html',
 'api/java.base/java/util/concurrent/atomic/AtomicLong.html',
 'api/java.base/java/util/concurrent/ConcurrentHashMap.html',
 'api/java.base/java/util/concurrent/Executor.html',
 'api/java.base/java/util/concurrent/atomic/AtomicReference.html',
 'api/java.base/java/util/concurrent/Flow.Subscription.html',
 'api/java.base/java/util/concurrent/Semaphore.html',
 'api/java.base/java/util/concurrent/DelayQueue.html',
 'api/java.base/java/util/concurrent/FutureTask.html',
 'api/java.base/java/lang/Runnable.html',
 'api/java.base/java/lang/Void.html',
 'api/java.base/java/util/concurrent/ThreadPoolExecutor.CallerRunsPolicy.html',
 'api/java.base/java/util/concurrent/ScheduledThreadPoolExecutor.html',
 'api/java.base/java/util/concurrent/LinkedBlockingQue